In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
 from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
import sys, os
import glob
import warnings
sys.path.append('../')

import numpy as np
import pandas

from sklearn.metrics import average_precision_score as aps

In [4]:
def color_value(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val < 0:
        color = 'red' 
    elif val > 0:
        color = 'blue'
    else:
        color = 'black'
    return 'color: %s' % color

In [5]:
def pairwise_rerank(labels):
    
    labels_ = labels.copy()
    order = np.arange(labels.size)
    
    i = 1
    while i+1 < labels.size:
        if labels_[i] == 0 and labels_[i+1] == 1:
            labels_[i+1] = 0
            labels_[i] = 1
            
            swp = order[i+1]
            order[i+1] = order[i]
            order[i] = swp
            i += 1
        else:
            i += 1
            
    return order

In [6]:
def iterative_rerank(pred, max_i):
    
    rr = np.arange(pred.shape[0])
    for i in range(max_i):
        
        norder = pairwise_rerank(pred)
        if not np.all(pred == pred[norder]):
            pred = pred[norder]
            rr = rr[norder]
        else:
            break
            
    return rr

In [7]:
def removal_rerank(pred):
    return

In [8]:
gt = np.load('/home/alberto/phD/projects/performance_prediction/ret-mr-learning/input-labels/label.v2/vggfaces/vggfaces.002.vgg16-pert-L2.irp_lbls.npy')
pred = np.load('/home/alberto/phD/projects/performance_prediction/ret-mr-learning/output/out.v2/vggfaces/002.vgg16-pert-L2/pos-learn.0006.irp.cid-jaccard-20-100.k30.svc/vggfaces_002_5rounds2splits_top30_irp.npy')

k = 30
eval_k = 10

gt = gt[:, 0:100]
pred = pred[0, :, 0:k]

n = gt.shape[0]
print(n)

7866


In [9]:
data_ap = {'previous AP':[],
        'new AP':[],
        'change':[]}#,
        #'change %':[]}
    
data_p = {'previous P':[],
        'new P':[],
        'change':[]}
        
for i in range(n):
    
    gt_label = gt[i]
    p_label = pred[i]
    
    v = aps(gt_label, np.arange(k, 0, -1), pos_label=1)
    if np.isnan(v):
        v = 0
    data_ap['previous AP'].append(v)
    data_p['previous P'].append(gt_label[0:eval_k].mean())
    
    rr = iterative_rerank(p_label, 10)
    
    
    v = aps(gt_label[rr], np.arange(k, 0, -1), pos_label=1)
    if np.isnan(v):
        v = 0
    data_ap['new AP'].append(v)
    
    data_ap['change'].append(data_ap['new AP'][-1] - data_ap['previous AP'][-1])
    
    
    
    aux = gt_label[rr]
    v = np.mean(aux[0:eval_k])
    data_p['new P'].append(v)
    data_p['change'].append(data_p['previous P'][-1] - data_p['previous P'][-1])
    #data_ap['change %'].append(data_ap['new AP'][-1]/data_ap['previous AP'][-1] - 1)

ValueError: Found input variables with inconsistent numbers of samples: [100, 30]

In [ ]:
pandas.options.display.float_format = '{:.4f}'.format
df = pandas.DataFrame(data_ap)
print(df.describe())
df.style.applymap(color_value, subset=['change'])

In [ ]:
pandas.options.display.float_format = '{:.4f}'.format
df = pandas.DataFrame(data_p)
print(df.describe())
df.style.applymap(color_value, subset=['change'])